## Daily Aggregrage
- dictionary is done
- aggreage both domain, tag and tag_gp

### 0. Initialization

In [1]:
from smtutil import common, hiveutil
from datetime import datetime, timedelta
import calendar
import os
import argparse
import shutil
import logging
import sys
import yaml
import pyspark.sql.functions as F
import glob

In [2]:
logger = logging.getLogger(__name__)
log_format = "[%(asctime)s][%(funcName)25s:%(lineno)4d] %(message)s"
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format=log_format)

In [3]:
# def main(t_domain_base_dir, spark=None):
    
#     # define sc for checking to closing spark
#     sc=False

#     # get spark
#     if not spark:
#         sc, spark = hiveutil.get_sc_spark()
#         logging.info("No spark session. Created one now")
        
#     ## ---------------------------- Path Settings ------------------------------ ##   
#     # ssl_data_dir = '/app/sna_bdmrdev01/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/'
#     t_domain_base_dir = '/app/sna_bdmrdev01/ws-ic/hive/weblog.db/'

#     ## path settings
#     t_domain_daily_dir = t_domain_base_dir + '/t_domain_daily/'
#     t_domain_monthly_dir = t_domain_base_dir + '/t_domain_monthly/'
#     # t_domain_index_dir = t_domain_base_dir + '/t_domain_index/'
    
#     ## ---------------------------- Get Sdfs ------------------------------ ##
#     sdf_tdd_cm = spark.read.parquet(t_domain_daily_dir).filter(F.col('date_id').between(date_id_st, date_id_end))
#     sdf_tdm = spark.read.parquet(t_domain_monthly_dir)
    
#     ## ------------------------ Aggregate ------------------------ ##
#     # get days of a month
#     count_days = sdf_tdd_cm.select('date_id').distinct().count()

#     # check if all t doamin daily ready and t domain monthly not exist
#     if (count_days == days) and (len(glob.glob(t_domain_monthly_dir + f'*={month_id}')) == 0):
        
#         logging.info('Ready to generate t domain monthly')
        
#         # aggregate
#         sdf_tdm_new = sdf_tdd_cm.groupby(['subr_num', 'agg_value', 'agg_level']) \
#                         .agg( 
#                             F.sum(F.col('in_byte')).alias('in_byte'),
#                             F.sum(F.col('out_byte')).alias('out_byte'),
#                             F.sum(F.col('byte_count')).alias('byte_count'),
#                             F.sum(F.col('total_hit')).alias('total_hit'),
#                             F.sum(F.col('duration')).alias('duration'),
#                             F.countDistinct('date_id').alias('days')) \
#                         .withColumn('month_id', F.lit(month_id))
        
#         # write
#         # no need to delete old directory since checked above
#         sdf_tdm_new.coalesce(10) \
#                    .write.mode('append') \
#                    .partitionBy('month_id') \
#                    .parquet(t_domain_monthly_dir)
        
#     # stop spark session and spark content
#     if sc == False:
#         spark.stop()
#         sc.stop()
#         logging.info("Stopped spark session")

#     logging.info(f"End, {datetime.now()}")
    
#     return

In [4]:
sc, spark = hiveutil.get_sc_spark()
hiveutil.register_common_udf(spark)

In [5]:
# get month id
month_id = int((datetime.now().replace(day=1) - timedelta(days=1)).strftime("%Y%m"))

# get date_id start and end
date_id_st = month_id * 100 + 1
date_id_end = month_id * 100 + calendar.monthrange(month_id // 100, month_id % 100)[1]
days = date_id_end - date_id_st + 1
print(date_id_st, date_id_end, days)

20230101 20230131 31


In [6]:
# ssl_data_dir = '/app/sna_bdmrdev01/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/'
t_domain_base_dir = '/app/sna_bdmrdev01/ws-ic/hive/weblog.db/'

## path settings
t_domain_daily_dir = t_domain_base_dir + '/t_domain_daily/'
t_domain_monthly_dir = t_domain_base_dir + '/t_domain_monthly/'
# t_domain_index_dir = t_domain_base_dir + '/t_domain_index/'

In [7]:
# list of data to loaded
# sdf_ssl = spark.read.parquet(ssl_data_dir)
sdf_tdd_cm = spark.read.parquet(t_domain_daily_dir).filter(F.col('date_id').between(date_id_st, date_id_end))
sdf_tdm = spark.read.parquet(t_domain_monthly_dir)
# sdf_tdi = spark.read.parquet(t_domain_index_dir)

In [ ]:
# get days of a month
count_days = sdf_tdd_cm.select('date_id').distinct().count()

# check if all t doamin daily ready and t domain monthly not exist
if (count_days == days) and (len(glob.glob(t_domain_monthly_dir + f'*={month_id}')) == 0):

    sdf_tdm_new = sdf_tdd_cm.groupby(['subr_num', 'agg_value', 'agg_level']) \
                        .agg( 
                            F.sum(F.col('in_byte')).alias('in_byte'),
                            F.sum(F.col('out_byte')).alias('out_byte'),
                            F.sum(F.col('byte_count')).alias('byte_count'),
                            F.sum(F.col('total_hit')).alias('total_hit'),
                            F.sum(F.col('duration')).alias('duration'),
                            F.countDistinct('date_id').alias('days')) \
                        .withColumn('month_id', F.lit(month_id))
    
    # write
    sdf_tdm_new.write.mode('append') \
               .partitionBy('month_id') \
               .parquet(t_domain_monthly_dir)


In [10]:
spark.stop()
sc.stop()

In [11]:
# sdf_tdm_new.limit(5).toPandas()

subr_num  agg_value agg_level  in_byte  out_byte  byte_count  total_hit  \
0  19594837      41541    domain        0         0    17729626        182   
1  46291789      32176    domain        0         0       46524          4   
2  51191571      56758    domain        0         0      505317         66   
3  51390544      31092    domain        0         0      723479         10   
4  51977266      57009    domain        0         0      556404         50   

   duration  days  month_id  
0         0    28    202301  
1         0     3    202301  
2         0    18    202301  
3         0     8    202301  
4         0    14    202301